In [1]:
import json
import string
import random 
import nltk
import numpy as num
from nltk.stem import WordNetLemmatizer 
import tensorflow as tensorF
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
# nltk.download("punkt")
# nltk.download("wordnet")
# nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\devlo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\devlo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\devlo\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [35]:
f=open('ourdata.json')
data=json.load(f)

In [36]:
print(data['10'])

{'Patterns': ['where can i get the add drop form?', 'where can i get that form', 'where can i get that form?', 'open elective change from Nonlinear Dynamics and Machine Learning in Complex Systems to Digital Marketing ', 'open elective change from Nonlinear Dynamics\nand Machine Learning\nin Complex Systems to digital marketing ', '\xa0I want to change my open elective subject\n\xa0I have selected digital marketing by mistake just now\n\xa0Can you help me change it to Web technologie', 'change open elective ', 'I completed my course registration on 10th January. Till then only four courses were being displayed including SEO which I chose. It was later that more subjects were added to choose from. I would like to drop SEO and take Web Technology CSET382.', 'Would like to change my Elective from Advanced Skill Enhancement to Sociology ', 'what is the procedure to drop minor?', 'I want to change my open elective from "Advanced skill enhancement" to "Web technology". Batch-EB05 (2021 BATCH

In [37]:
f=0
for intents in data:
  print(f)
  print(data[intents]['Responses'][0])
  f+=1

0
Kindly add the screenshot of your result which is reflected on your screen and also the registration slip
1
Dear Student,

as per your request your course has been dropped
2
Dear Student,  Kindly share the query.
3
Dear student your batch changing request has been updated on collpoll, kindly check
4
Dear Student, it is not possible now as the deadline is already crossed
5
Window is already closed, you can again try your luck in the next semester.
6
Dear Student,  add/drop request you need to send on CollPoll.
7
Dear Student, I have already replied to this issue. So i m closing this quary. Regards
8
Dear Diven, Kindly mail to c-saphal.anand@bennett.edu.in  or  Saphal Anand <saphal.a@collpoll.com> for all your queries regarding collpoll to get resolved
9
Dear Student , knidly contact with your faculty mentor
10
Dear Student,
you can get the form in the mail sent by Registrar before the registration in Early August.
there will be a dccument consisting of multiple things and the form is 

In [38]:

nltk.download('stopwords')
#nltk.download('wordnet')


import string
def text_process(text):

    stemmer = WordNetLemmatizer()
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join([i for i in nopunc if not i.isdigit()])
    nopunc =  [word.lower() for word in nopunc.split() if word not in stopwords.words('english')]
    return [stemmer.lemmatize(word) for word in nopunc]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\devlo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stopwords.words('english').append(x for x in ['dear','student','students','Please','please','sir','dear'])

In [40]:
lm = WordNetLemmatizer()
ourClasses = []
newWords = []
documentX = []
documentY = []
for intent in data.keys():
    for pattern in data[intent]['Patterns']:
        ourTokens = nltk.word_tokenize(pattern)
        newWords.extend(ourTokens)
        documentX.append(pattern)
        documentY.append(intent)

    if intent not in ourClasses:# add unexisting tags to their respective classes
        ourClasses.append(intent)
newWords = [lm.lemmatize(word.lower()) for word in newWords if word not in string.punctuation if word not in stopwords.words('english')] # set words to lowercase if not in punctuation
newWords = sorted(set(newWords))# sorting words
ourClasses = sorted(set(ourClasses))# sorting classes

In [41]:
newWords

["''",
 "'acad",
 "'batch",
 "'computational",
 "'d",
 "'engineering",
 "'ll",
 "'m",
 "'s",
 "'ve",
 '+91',
 '-probability',
 '..',
 '...',
 '....',
 '.i',
 '.please',
 '.sir',
 '//webapp.csebu.com/',
 '0',
 '01',
 '03',
 '1',
 '1,23,750',
 '1/7',
 '10',
 '100',
 '103',
 '103°',
 '107',
 '108',
 '10th',
 '11',
 '11-03-2003',
 '12',
 '12/08/2022',
 '13',
 '13th',
 '14',
 '15',
 '16',
 '16th',
 '17',
 '17th',
 '18',
 '1st',
 '2',
 '20',
 '2020',
 '2021',
 '2021-25',
 '2022',
 '2023',
 '2023.so',
 '2025',
 '208007',
 '20th',
 '21',
 '21/08',
 '22',
 '24',
 '24,25,30',
 '24,25nov.',
 '240',
 '25',
 '25nov',
 '26',
 '26credits',
 '28',
 '29',
 '29th',
 '2nd',
 '3',
 '3-0-0',
 '3-0-2',
 '30',
 '30th',
 '33',
 '34',
 '35',
 '36',
 '382',
 '3components',
 '3credits',
 '3rd',
 '4',
 '4-5',
 '41',
 '42',
 '45',
 '45546',
 '46',
 '489',
 '49',
 '492',
 '4th',
 '5',
 '50',
 '50+',
 '50-60',
 '51',
 '52',
 '5pm',
 '6',
 '6/60',
 '6am',
 '6pm',
 '6th',
 '70',
 '7:40pm',
 '7th',
 '8',
 '8920237098',

In [42]:
trainingData = []
s="" 
outEmpty = [0] * len(ourClasses)
for idx, doc in enumerate(documentX):
    bagOfwords = []
    s=""
    k=doc.split()
    for x in k:
        x=x.lower()
        if x not in string.punctuation and x not in stopwords.words('english'):
            s+=x+" "
    text = lm.lemmatize(s)
    for word in newWords:
        bagOfwords.append(1) if word in text else bagOfwords.append(0)

    outputRow = list(outEmpty)
    outputRow[ourClasses.index(documentY[idx])] = 1
    trainingData.append([bagOfwords, outputRow])

random.shuffle(trainingData)
trainingData = num.array(trainingData, dtype=object)

x = num.array(list(trainingData[:, 0]))
y = num.array(list(trainingData[:, 1]))

KeyboardInterrupt: 

In [ ]:
iShape = (len(x[0]),)
oShape = len(y[0])

ourNewModel = Sequential()
ourNewModel.add(Dense(128, input_shape=iShape, activation="relu"))

ourNewModel.add(Dense(64, activation="relu"))
#ourNewModel.add(Dropout(0.3))
ourNewModel.add(Dense(oShape, activation = "softmax"))
md = tensorF.keras.optimizers.Adam()
ourNewModel.compile(loss='categorical_crossentropy',
              optimizer=md,
              metrics=["accuracy"])
print(ourNewModel.summary())
ourNewModel.fit(x, y, epochs=400, verbose=1)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 128)               212864    
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dense_8 (Dense)             (None, 15)                975       
                                                                 
Total params: 222,095
Trainable params: 222,095
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/400
19/19 [==============================] - 1s 4ms/step - loss: 2.4236 - accuracy: 0.2325
Epoch 2/400
19/19 [==============================] - 0s 4ms/step - loss: 2.0318 - accuracy: 0.3983
Epoch 3/400
19/19 [==============================] - 0s 4ms/step - loss: 1.8266 - accuracy: 0.4359
Epoch 4/40

In [ ]:
l=[x for x in data.keys()]
l

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26']

In [ ]:
def ourText(text):
  newtkns = nltk.word_tokenize(text)
  newtkns = [lm.lemmatize(word.lower()) for word in newtkns if word not in string.punctuation if word not in stopwords.words('english')]
  return newtkns

def wordBag(text, vocab):
  newtkns = ourText(text)
  bagOwords = [0] * len(vocab)
  for w in newtkns:
    for idx, word in enumerate(vocab):
      if word == w:
        bagOwords[idx] = 1
  return num.array(bagOwords)

def Pclass(text, vocab, labels):
  bagOwords = wordBag(text, vocab)
  ourResult = ourNewModel.predict(num.array([bagOwords]))[0]
  newThresh = 0.2
  yp = [[idx, res] for idx, res in enumerate(ourResult) if res > newThresh]

  yp.sort(key=lambda x: x[1], reverse=True)
  newList = []
  for r in yp:
    newList.append(labels[r[0]])
  return newList

def getRes(firstlist, data):
  brr=" "
  tag = firstlist[0]
  listOfIntents = [x for x in data.keys()]
  for i in listOfIntents:
    if i == tag:
      #print("waaw")
      brr = random.choice(data[i]['Responses'])
      break
  return brr

In [ ]:
while x<3:
    newMessage = input()
    intents = Pclass(newMessage, newWords, ourClasses)
    print(getRes(intents, data))
    x+=1

1/1 [==============================] - 0s 121ms/step
kindly refer to the mail shared by dean Sir today regarding batch change
1/1 [==============================] - 0s 30ms/step
Dear Student, I didnt found any minor course in your registrated courses. Kindly check again.
1/1 [==============================] - 0s 27ms/step
Dear Kartike, As per UGC guidelines, 75% attendance is required for each course to qualify to sit in the final exam. This 75% is calculated by adding labs, Tutorials and lectures of the individual courses. This buffer of 25% contains coverage of all situa
1/1 [==============================] - 0s 35ms/step
Dear Student, this is due to technical glithch and it has raised to collpoll. if it is not adressed yet then kindly put a reminder to it so that it can be escalated
1/1 [==============================] - 0s 29ms/step
Dear Student, try following ways to resolve the issue ; clear your browsing history/cookies and try after that OR try using a different browser.
1/1 [=

KeyboardInterrupt: Interrupted by user